# Information Assistant with LangGraph - Testing Notebook

This notebook tests the functionality of the Information Assistant system built with LangGraph. The assistant provides:

- 🎧 **Podcast Recommendations**: Find podcasts based on interests, topics, or similar to existing podcasts
- 📰 **News Updates**: Get recent news on various topics
- 💬 **Conversational Interface**: Natural language interaction with context retention

## Setup

First, let's make sure we have the required packages installed:

In [1]:
# Install required packages
!pip install langchain langchain_openai tavily-python python-dotenv requests langgraph


[notice] A new release of pip is available: 24.0 -> 25.1.1
[notice] To update, run: pip install --upgrade pip


## Environment Setup

Set up your API keys in a .env file or directly here:

In [ ]:
import os
from dotenv import load_dotenv

# Load environment variables from .env file
load_dotenv()

# Check if API keys are set
tavily_key = os.getenv("TAVILY_API_KEY")
openai_key = os.getenv("OPENAI_API_KEY")

print(f"Tavily API Key: {'✓ Set' if tavily_key else '✗ Not Set'}")
print(f"OpenAI API Key: {'✓ Set' if openai_key else '✗ Not Set'}")

Tavily API Key: ✓ Set
OpenAI API Key: ✓ Set


# Part 1: Testing Individual Tools

Before we test the full LangGraph workflow, let's test the individual tools to ensure they work correctly.

## Podcast Tools

Let's test the podcast recommendation tools first.

In [ ]:
# Import podcast tools from the tools directory
from tools.podcast_tools import (
    get_podcast_recommendations,
    get_podcast_details,
    get_similar_podcasts,
    get_topic_podcasts,
    podcast_tools
)

In [ ]:
# Test podcast recommendations
print("Testing podcast recommendations...\n")

results = get_podcast_recommendations(
    query="educational history podcasts about ancient Rome",
    max_results=3,
    include_episodes=True
)

# Display results
print(f"Found {results['recommendation_count']} recommendations for '{results['original_query']}'\n")

for i, podcast in enumerate(results.get('recommendations', []), 1):
    print(f"===== PODCAST {i} =====")
    print(f"Name: {podcast.get('name', 'Unknown')}")
    print(f"Publisher: {podcast.get('publisher', 'Unknown')}")
    print(f"Description: {podcast.get('description', 'No description available')[:150]}...")
    
    # Show episodes if available
    episodes = podcast.get('episodes', [])
    if episodes:
        print("\nRecent episodes:")
        for j, episode in enumerate(episodes[:2], 1):
            print(f"  {j}. {episode.get('title', 'Untitled')}")
    print("\n")

In [ ]:
# Test get podcast details
print("Testing podcast details lookup...\n")

# Get details about a specific podcast (using Hardcore History as an example)
details = get_podcast_details(podcast_name="Hardcore History")

# Display details
if details:
    podcast = details.get('podcast', {})
    print(f"Title: {podcast.get('title', 'Unknown')}")
    print(f"Publisher: {podcast.get('publisher', 'Unknown')}")
    print(f"Description: {podcast.get('description', 'No description')[:200]}...\n")
    
    episodes = podcast.get('episodes', [])
    if episodes:
        print("Recent episodes:")
        for i, episode in enumerate(episodes[:2], 1):
            print(f"  {i}. {episode.get('title', 'Untitled')}")
else:
    print("No details found")

## News Tools

Now let's test the news tools functionality.

In [ ]:
# Import news tools
from tools.news_tools import get_recent_news

# Test news search
print("Testing news search...\n")

topic = "artificial intelligence"
days_back = 7

news_results = get_recent_news(topic=topic, days_back=days_back)

# Display results
print(f"Found {news_results.get('article_count', 0)} news articles about '{topic}' in the last {days_back} days\n")

print("Summary:")
print(news_results.get('summary', 'No summary available'))
print("\nArticles:")

for i, article in enumerate(news_results.get('articles', []), 1):
    print(f"\n{i}. {article.get('title', 'No title')}")
    print(f"   Source: {article.get('source', 'Unknown')}")
    print(f"   Date: {article.get('date', 'Unknown date')}")
    print(f"   {article.get('snippet', 'No snippet available')[:150]}...")

# Part 2: Testing the Full LangGraph Workflow

Now let's test the complete LangGraph workflow that integrates all tools together.

In [ ]:
# Import the main workflow
from main import run_info_assistant
from graph.workflow import create_info_assistant
from graph.state import InfoAssistantState
from langchain_core.messages import HumanMessage, AIMessage

# Also import visualization libraries
import sys
import platform

# Check if graphviz is available
try:
    from graphviz import Digraph
    graphviz_available = True
except ImportError:
    graphviz_available = False
    print("Graphviz not installed. Some visualizations may not be available.")
    print("Install graphviz using your package manager and pip install graphviz")

In [ ]:
# Test basic workflow with a podcast query
print("Testing workflow with podcast query...\n")

podcast_query = "Can you recommend some good history podcasts?"
podcast_response = run_info_assistant(podcast_query)

print(f"Query: {podcast_query}")
print(f"\nResponse:\n{podcast_response}")

In [ ]:
# Test workflow with a news query
print("Testing workflow with news query...\n")

news_query = "What's the latest news about space exploration?"
news_response = run_info_assistant(news_query)

print(f"Query: {news_query}")
print(f"\nResponse:\n{news_response}")

# Testing Multi-turn Conversation

Let's test a multi-turn conversation to see how the assistant maintains context:

In [ ]:
# Create a conversation ID for this session
import uuid
conversation_id = str(uuid.uuid4())
print(f"Starting conversation with ID: {conversation_id}\n")

# First query
query1 = "I'm interested in learning about ancient Rome. Any good podcasts?"
response1 = run_info_assistant(query1, conversation_id=conversation_id)

print(f"User: {query1}\n")
print(f"Assistant: {response1}\n")

In [ ]:
# Follow-up query referring to the previous response
query2 = "Can you tell me more about the first podcast you mentioned?"
response2 = run_info_assistant(query2, conversation_id=conversation_id)

print(f"User: {query2}\n")
print(f"Assistant: {response2}\n")

In [ ]:
# Now change topic to news
query3 = "What's been happening in the news about artificial intelligence lately?"
response3 = run_info_assistant(query3, conversation_id=conversation_id)

print(f"User: {query3}\n")
print(f"Assistant: {response3}\n")

In [ ]:
# Follow up on news
query4 = "Any developments related to AI regulation?"
response4 = run_info_assistant(query4, conversation_id=conversation_id)

print(f"User: {query4}\n")
print(f"Assistant: {response4}\n")

# Visualize the LangGraph Workflow

Let's visualize the structure of our LangGraph workflow:

In [ ]:
def visualize_info_assistant_graph():
    """Create a visualization of the LangGraph workflow"""
    if not graphviz_available:
        from IPython.display import HTML
        return HTML("""
        <div style="text-align:center; padding:20px; border:1px solid #ddd; border-radius:5px;">
            <p><b>Graphviz not available</b></p>
            <p>Install graphviz to see workflow visualization</p>
        </div>
        """)
        
    # Create visualization
    dot = Digraph(comment='Info Assistant Workflow')
    dot.attr(rankdir='TB')  # Top to bottom layout
    
    # Define node styles
    dot.attr('node', shape='box', style='filled', fontname='Arial')
    
    # Add nodes
    dot.node('main_agent', 'Main Agent\n(Router)', fillcolor='lightblue')
    dot.node('podcast_tools', 'Podcast Tools', fillcolor='lightyellow')
    dot.node('news_tools', 'News Tools', fillcolor='lightgreen')
    dot.node('respond', 'Response Handler', fillcolor='lightpink')
    dot.node('END', 'END', shape='doublecircle', fillcolor='lightgray')
    
    # Add edges
    dot.edge('main_agent', 'podcast_tools', label='podcast query')
    dot.edge('main_agent', 'news_tools', label='news query')
    dot.edge('main_agent', 'respond', label='direct response')
    dot.edge('main_agent', 'END', label='end conversation')
    dot.edge('podcast_tools', 'respond')
    dot.edge('news_tools', 'respond')
    dot.edge('respond', 'main_agent', label='next turn')
    
    return dot

# Display the visualization
graph = visualize_info_assistant_graph()
if graphviz_available:
    display(graph)

# Summary

This notebook has demonstrated:

1. Testing the podcast recommendation tools
2. Testing the news search tools
3. Testing the full LangGraph workflow that integrates both capabilities
4. Multi-turn conversations that maintain context
5. Visualization of the workflow architecture

The Information Assistant successfully combines podcast recommendations and news updates in a conversational interface powered by LangGraph's workflow capabilities.